In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Input, Flatten, Conv2D, Activation, MaxPooling2D, Dropout, GlobalMaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD, Adam, Adadelta
from tensorflow.keras.callbacks import History, EarlyStopping

import matplotlib.pyplot as plt

In [2]:
# Función que nos retorna las clases del df introducido como si fueran 10 submuestras por cada señal
def aument_dataframe_clases(df) -> list:
    data = []
    for clase in df['Class']:
        for i in range(0, 10):
            data.append(clase)

    df = pd.DataFrame(data, columns =['Class'], dtype = str)

    return df['Class']

In [3]:
# Cargamos la matriz de la iteración que corresponda
def load_matrix(i):
    
    path_train = 'Clasificaciones/train_' + str(i) + '.csv'
    path_valid = 'Clasificaciones/validation_' + str(i) + '.csv'
    
    df_y_t = pd.read_csv(path_train)
    df_y_v = pd.read_csv(path_valid)
    
    X_train = np.load("Variables/matrix_train.npy") # cargar la matriz guardada
    X_train = X_train.reshape(4, len(df_y_t)*10, 100, 121)
    
    y_train = aument_dataframe_clases(df_y_t) # cargar la matriz guardada y quedarse con la columna de clase
    y_train[y_train == 'No jamming'] = 0
    y_train[y_train == 'Jamming'] = 1
    y_train = to_categorical(y_train)

    X_val = np.load("Variables/matrix_valid.npy") # cargar la matriz guardada
    X_val = X_val.reshape(4, len(df_y_v)*10, 100, 121)
    
    y_val = aument_dataframe_clases(df_y_v) # cargar la matriz guardada y quedarse con la columna de clase
    y_val[y_val == 'No jamming'] = 0
    y_val[y_val == 'Jamming'] = 1
    y_val = to_categorical(y_val)
    
    return X_train, y_train, X_val, y_val

In [4]:
def model_creator_old():
    
    feature_vector_length = 121 # 2*60 + 1
    num_classes = 2

    input_shape = (100, feature_vector_length, 1)
    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

    model.add(Conv2D(64, (3, 3),padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

    model.add(Conv2D(520, (3, 3),padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))


    model.add(Flatten())  
    model.add(Dense(520))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('sigmoid'))
    
    return model

In [7]:
old_model = model_creator_old()
print(old_model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 98, 119, 32)       320       
_________________________________________________________________
activation_4 (Activation)    (None, 98, 119, 32)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 49, 60, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 49, 60, 64)        18496     
_________________________________________________________________
activation_5 (Activation)    (None, 49, 60, 64)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 25, 30, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 25, 30, 520)      

In [4]:
def model_creator():
    
    feature_vector_length = 121 # 2*60 + 1
    num_classes = 2

    input_shape = (100, feature_vector_length, 1)
    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

    model.add(Conv2D(64, (3, 3),padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

    model.add(Conv2D(520, (3, 3),padding='same'))
    model.add(Activation('relu'))

    model.add(GlobalMaxPooling2D())
    model.add(Dense(num_classes))
    model.add(Activation('sigmoid'))
    
    return model

In [6]:
model = model_creator()
print(model.summary())

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 119, 32)       320       
_________________________________________________________________
activation (Activation)      (None, 98, 119, 32)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 60, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 49, 60, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 49, 60, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 30, 64)        0         
_________________________________________

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='new_model_plot.png', show_shapes=True, show_layer_names=True)

In [22]:
# Configuramos el modelo y empezamos a entrenarlo (cross val)
def model_training_cross_val(model, optimizer, epochs, batch_size, i):
    
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    # Cargamos los datos de entrenamiento y validación según iteración
    X_train, y_train, X_val, y_val = load_matrix(i)

    X_train = np.expand_dims(X_train[i], axis=3)
    X_val   = np.expand_dims(X_val[i], axis=3)

    history  = model.fit(X_train, y_train, epochs=epochs, 
                       batch_size=batch_size, verbose=1, 
                       callbacks=[EarlyStopping(monitor='val_loss', verbose=1, patience=20)], 
                       validation_data=(X_val, y_val))
    
    # Ploteamos la curva de valores del accuracy y loss
    plot_model_values_cross(history.history['acc'], history.history['val_acc'], history.history['loss'], history.history['val_loss'])
        
    return history

In [38]:
# Entrenamiento final del modelo
def final_model_training(model, optimizer, epochs, batch_size):
    
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    # Cargamos los datos de entrenamiento y validación según iteración
    X_train, y_train, X_val, y_val = load_matrix(0)
    
    X_train = np.concatenate((X_train[0], X_val[0]), axis=0)
    y_train = np.concatenate((y_train, y_val), axis=0)

    X_train = np.expand_dims(X_train, axis=3)
    
    history  = model.fit(X_train, y_train, epochs=epochs, 
                       batch_size=batch_size, verbose=1, 
                       callbacks=[EarlyStopping(monitor='loss', verbose=1, patience=20)])
    
    # Ploteamos la curva de valores del accuracy y loss
    plot_model_values(history.history['acc'], history.history['loss'])
        
    return history

In [4]:
def plot_model_values_cross(hist_acc, hist_val_acc, hist_loss, hist_val_loss):
        
    plt.plot(hist_acc)
    plt.plot(hist_val_acc)
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

    plt.plot(hist_loss)
    plt.plot(hist_val_loss)
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

In [5]:
def plot_model_values(hist_acc, hist_loss):
        
    plt.plot(hist_acc)
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train'], loc='upper left')
    plt.show()

    plt.plot(hist_loss)
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train'], loc='upper left')
    plt.show()

In [ ]:
# Instanciamos el modelo y lo entrenamos 4 veces
lr = 0.001
optimizer = Adadelta(learning_rate = lr)

epochs = 500
batch_size = 32

for i in range (0, 4):
    model = model_creator()
    history = model_training_cross_val(model, optimizer, epochs, batch_size, i)

    filename = "Nuevos_modelos/adadelta_model_lr_" + str(lr) + "_" + str(i)
    model.save(filename)
    
    del model
    del history

In [ ]:
# Instanciamos el modelo y lo entrenamos con todos los valores
model = model_creator()
history = final_model_training(model, optimizer, epochs, batch_size)

filename = "Nuevos_modelos/adadelta_model_lr_" + str(lr) + "_final"
model.save(filename)
    
del model
del history

In [6]:
#Fine tuning - transfer learning con modelo VGG16
def vgg_model_creator():
    vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(100, 121, 3))
    
    for layer in vgg_model.layers:
        layer.trainable = False

    output = vgg_model.layers[-1].output
    
    x = GlobalMaxPooling2D()(output)
    out = Dense(2, activation = 'sigmoid')(x)
    
    model = Model(inputs = vgg_model.inputs,
                  outputs = out,
                  name = 'vgg16_model')
       
    return model

In [7]:
from tensorflow.keras.applications.vgg16 import VGG16

vgg16_model = vgg_model_creator()
vgg16_model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "vgg16_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 121, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 121, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 121, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 60, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 60, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 60, 128)       147584    
________________________________________

In [8]:
# Configuramos el modelo y empezamos a entrenarlo (cross validation)
def model_training_fine_tuning_cross_val(model, optimizer, epochs, batch_size, i):
    
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    # Cargamos los datos de entrenamiento y validación según iteración
    X_train, y_train, X_val, y_val = load_matrix(i)

    X_train = np.expand_dims(X_train[i], axis=3)
    X_val   = np.expand_dims(X_val[i], axis=3)

    # Pasamos a tener 3 dimensiones para que el modelo pueda funcionar
    X_train3d = np.tile(X_train, 3)
    X_val3d = np.tile(X_val, 3)

    history  = model.fit(X_train3d, y_train, epochs=epochs, 
                       batch_size=batch_size, verbose=1, 
                       callbacks=[EarlyStopping(monitor='val_loss', verbose=1, patience=20)], 
                       validation_data=(X_val3d, y_val))
    
    # Ploteamos la curva de valores del accuracy y loss
    plot_model_values_cross(history.history['acc'], history.history['val_acc'], history.history['loss'], history.history['val_loss'])
        
    return history

In [9]:
# Entrenamiento final
def final_model_training_fine_tuning(model, optimizer, epochs, batch_size):
    
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    # Cargamos los datos de entrenamiento y validación según iteración
    X_train, y_train, X_val, y_val = load_matrix(0)
    
    X_train = np.concatenate((X_train[0], X_val[0]), axis=0)
    y_train = np.concatenate((y_train, y_val), axis=0)

    X_train = np.expand_dims(X_train, axis=3)
        
    # Pasamos a tener 3 dimensiones para que el modelo pueda funcionar
    X_train3d = np.tile(X_train, 3)

    history  = model.fit(X_train3d, y_train, epochs=epochs, 
                       batch_size=batch_size, verbose=1, 
                       callbacks=[EarlyStopping(monitor='loss', verbose=1, patience=20)])
    
    # Ploteamos la curva de valores del accuracy y loss
    plot_model_values(history.history['acc'], history.history['loss'])
        
    return history

In [ ]:
# Instanciamos el modelo y lo entrenamos 4 veces

del vgg16_model

lr = 0.01
optimizer = Adadelta(learning_rate = lr)

epochs = 500
batch_size = 32

for i in range (0, 4):
    vgg16_model = vgg_model_creator()
    history = model_training_fine_tuning_cross_val(vgg16_model, optimizer, epochs, batch_size, i)

    filename = "Nuevos_modelos/finetuning_adadelta_model_lr_" + str(lr) + "_" + str(i)
    vgg16_model.save(filename)
    
    del vgg16_model
    del history

ERROR! Session/line number was not unique in database. History logging moved to new session 988
Train on 360 samples, validate on 120 samples
Epoch 1/500
360/360 [==============================] - 8s 23ms/sample - loss: 11.0405 - acc: 0.5431 - val_loss: 14.6324 - val_acc: 0.4583
Epoch 2/500
360/360 [==============================] - 8s 23ms/sample - loss: 10.8515 - acc: 0.5472 - val_loss: 14.3737 - val_acc: 0.4667
Epoch 3/500
360/360 [==============================] - 9s 24ms/sample - loss: 10.6629 - acc: 0.5597 - val_loss: 14.1367 - val_acc: 0.4792
Epoch 4/500
360/360 [==============================] - 9s 25ms/sample - loss: 10.4871 - acc: 0.5611 - val_loss: 13.8866 - val_acc: 0.4875
Epoch 5/500
360/360 [==============================] - 9s 24ms/sample - loss: 10.3036 - acc: 0.5653 - val_loss: 13.6559 - val_acc: 0.5083
Epoch 6/500
360/360 [==============================] - 8s 24ms/sample - loss: 147.2408 - acc: 0.5653 - val_loss: 13.4407 - val_acc: 0.5125
Epoch 7/500
360/360 [========

In [ ]:
# Entrenamiento final con todos los parámetros
vgg16_model = vgg_model_creator()
history = final_model_training_fine_tuning(vgg16_model, optimizer, epochs, batch_size)

filename = "Nuevos_modelos/finetuning_adadelta_model_lr_" + str(lr) + "_final"
vgg16_model.save(filename)